In [ ]:
import os
import io
import time
import tarfile
import requests
from db import DB
from dotenv import load_dotenv

load_dotenv()

In [ ]:
db = DB()

### Signup test_jobcreator

In [ ]:
async def signin_signup():
    try: 
        token, user_id = await db.signin(
            {
                'username': os.getenv('DB_USERNAME'),
                'password': os.getenv('DB_PASSWORD'),
            }
        )
    except Exception as e: 
        token, user_id = await db.signup(
                {
                    'name': 'test',
                    'username': os.getenv('DB_USERNAME'),
                    'password': os.getenv('DB_PASSWORD'),
                }
            )
    return token, user_id

token, user_id = await signin_signup()

### List Nodes by Job Creator

In [ ]:
await db.list_nodes(
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)

### Add a job

In [ ]:
job_dict = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'docker_params': {
        'docker_image': 'sum-app',
        'docker_command':'python sum.py --num1 3 --num2 4',
        'docker_env_vars': {'env1': 'val1', 'env2': 'val2'}
    },
}

job = await db.create_job(
    job=job_dict,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

j

In [ ]:
print(j['output'])

### Add job with output folder

In [ ]:
job_dict = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'docker_params': {
        'docker_image': 'cv2-image',
        'docker_command':'python inference.py --prompt daimonxdaimon',
        'docker_output_dir': '/app/output',
    }
}

job = await db.create_job(
    job=job_dict,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

j

In [ ]:
print(j['output'])

### Get the output folder

In [ ]:
headers = {"Authorization": f"Bearer {token}"}
payload = {"id": job[1][0]['id']}

url = "http://localhost:8001/hub_get_job_output"

r = requests.post(url, headers=headers, json=payload)

In [ ]:
# Response is a tar folder; untar it and save it to a given location
output_dir = "/Users/arshath/play/daimon/output_folder/aaaa"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with tarfile.open(fileobj=io.BytesIO(r.content), mode="r|gz") as tar:
    tar.extractall(output_dir)

### Execute template job

In [ ]:
template_job = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'job_type': 'template',
    'template_params': {
        'template_name': 'hello-world-1',
        'template_args': {
        'name': 'template name',
        'description': 'template description',
        'param1': 3,
        'param2': 4,
        }
    }
}

job = await db.create_job(
    job=template_job,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

if j['status'] == 'completed':
    print(f"Job Status: {j['status']}")
    print(f"Job Output: {j['output']}")
else:
    print(f"Error: {j['error']} Error Message: {j['error_message']}")

In [ ]:
template_job = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'job_type': 'template',
    'template_params': {
        'template_name': 'hello-world-2',
        'template_args': {
        'name': 'template name',
        'description': 'template description',
        'param1': 10,
        'param2': 4,
        }
    }
}

job = await db.create_job(
    job=template_job,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

if j['status'] == 'completed':
    print(j['output'])
else:
    print(j['error_message'])

### Template - Research Assistant

In [ ]:
template_job = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'job_type': 'template',
    'template_params': {
        'template_name': 'research-assistant',
        'template_args': {
            'question': 'What is the some popular attractions in Singapore?',
        }
    }
}

job = await db.create_job(
    job=template_job,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

if j['status'] == 'completed':
    print(j['output'])
else:
    print(j['error_message'])

### Skeleton of thought

In [ ]:
template_job = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'job_type': 'template',
    'template_params': {
        'template_name': 'skeleton-of-thought',
        'template_args': {
            'question': 'What is the some popular attractions in Singapore?',
        }
    }
}

job = await db.create_job(
    job=template_job,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

if j['status'] == 'completed':
    print(j['output'])
else:
    print(j['error_message'])

### run_ollama

In [ ]:
template_job = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'job_type': 'template',
    'template_params': {
        'template_name': 'run-ollama',
        'template_args': {
            'model': 'phi',
            'prompt': 'tell me a joke',
        }
    }
}

job = await db.create_job(
    job=template_job,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

if j['status'] == 'completed':
    print(j['output'])
else:
    print(j['error_message'])

#### run_ollama with system_message

In [ ]:
template_job = {
    'owner': user_id,
    'name': 'test_job',
    'desc': 'test description',
    'job_type': 'template',
    'template_params': {
        'template_name': 'run-ollama',
        'template_args': {
            'model': 'mistral',
            'prompt': 'tell me a joke about yourself',
            'system_message': 'You are Ollama'
        }
    }
}

job = await db.create_job(
    job=template_job,
    username=os.getenv('DB_USERNAME'),
    password=os.getenv('DB_PASSWORD'),
)
job

In [ ]:
while True:
    j = await db.get_job(
        job_id=job[1][0]['id'],
        username=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
    )
    status = j['status']
    print(status)   

    if status == 'completed':
        break
    if status == 'error':
        break

    time.sleep(3)

if j['status'] == 'completed':
    print(j['output'])
else:
    print(j['error_message'])